In [1]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [2]:
n=10000
x=np.arange(n)
y = np.random.normal(size=(1000,512))+1

In [3]:
y.shape

(1000, 512)

In [4]:
kernel = stats.gaussian_kde(y)

NameError: name 'stats' is not defined

In [5]:
sns.histplot(y)

<AxesSubplot:ylabel='Count'>

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


Error in callback <function flush_figures at 0x7f68a5c4f700> (for post_execute):


KeyboardInterrupt: 

In [ ]:
ymin=min(y)
ymax=max(y)
ymin,ymax

In [ ]:
points=np.linspace(ymin,ymax,2000)

In [6]:
from scipy import stats

In [ ]:
x2=np.random.normal(size=(2,512))

In [ ]:
y2=kernel(x2)

In [ ]:
plt.plot(points,y2)